In [1]:
# Init spark session to read data from parquet files
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, IntegerType
from pyspark.sql import functions as F
from stocksx.configs.spark_config import SparkConfig
from stocksx.data_pipeline.sub_modules.spark_manager import SparkManager


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F

spark_config = SparkConfig(iceberg_enabled=True)
spark = SparkManager(spark_config)

In [10]:
# List all available catalogs
spark.session.sql("SHOW CATALOGS").show()

# List all namespaces in the local catalog
spark.session.sql("SHOW NAMESPACES IN spark_catalog").show()

+-------------+
|      catalog|
+-------------+
|        local|
|spark_catalog|
+-------------+

+---------+
|namespace|
+---------+
|  default|
+---------+



In [11]:
# Sanity check for a few symbols in the iceberg table

from pyspark.sql.functions import col

symbols_to_check = ["AAPL", "MSFT", "GOOGL"]

# Load data from the local Iceberg warehouse
# Using the catalog.database.table format
sample_df = spark.session.read.format("iceberg").table("spark_catalog.default.stock_prices")

# Alternatively, load directly from the file path
# sample_df = spark.read.format("iceberg").load("file:///e:/Projects/StocksX_Price_and_News_Influences/stocksx/data/iceberg_warehouse/default/table1")

# Filter for specific symbols
sample_df = sample_df.filter(col("symbol").isin(symbols_to_check))

# Display basic info about the data
print(f"Total records for symbols {symbols_to_check}: {sample_df.count()}")

# Check for missing values in important columns
print("\nMissing values in key columns:")
for column in sample_df.columns:
    null_count = sample_df.filter(col(column).isNull()).count()
    if null_count > 0:
        print(f"- {column}: {null_count} nulls")

# Show sample data for each symbol
print("\nSample data for each symbol:")
for symbol in symbols_to_check:
    print(f"\n--- {symbol} ---")
    sample_df.filter(col("symbol") == symbol).show(5, truncate=False)

# Basic statistics for numeric columns (assuming 'price' column exists)
if "price" in sample_df.columns:
    print("\nPrice statistics by symbol:")
    sample_df.groupBy("symbol").agg(
        min("price").alias("min_price"),
        max("price").alias("max_price"),
        avg("price").alias("avg_price"),
        count("price").alias("price_count")
    ).show()

Py4JJavaError: An error occurred while calling o111.table.
: org.apache.iceberg.hive.RuntimeMetaException: Failed to connect to Hive Metastore
	at org.apache.iceberg.hive.HiveClientPool.newClient(HiveClientPool.java:85)
	at org.apache.iceberg.hive.HiveClientPool.newClient(HiveClientPool.java:34)
	at org.apache.iceberg.ClientPoolImpl.get(ClientPoolImpl.java:143)
	at org.apache.iceberg.ClientPoolImpl.run(ClientPoolImpl.java:70)
	at org.apache.iceberg.ClientPoolImpl.run(ClientPoolImpl.java:65)
	at org.apache.iceberg.hive.CachedClientPool.run(CachedClientPool.java:122)
	at org.apache.iceberg.hive.HiveTableOperations.doRefresh(HiveTableOperations.java:147)
	at org.apache.iceberg.BaseMetastoreTableOperations.refresh(BaseMetastoreTableOperations.java:87)
	at org.apache.iceberg.BaseMetastoreTableOperations.current(BaseMetastoreTableOperations.java:70)
	at org.apache.iceberg.BaseMetastoreCatalog.loadTable(BaseMetastoreCatalog.java:49)
	at org.apache.iceberg.shaded.com.github.benmanes